## Import Libraries

In [1]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yup (use `wandb login --relogin` to force relogin)


True

In [2]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision import models

import matplotlib.pyplot as plt
%matplotlib inline

## Set Arguments & hyperparameters

In [3]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [4]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= './data'
NUM_CLS = 2

EPOCHS = 40
BATCH_SIZE = 32
LEARNING_RATE = 5*1e-4
EARLY_STOPPING_PATIENCE = 15
INPUT_SHAPE = 255

os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataloader

#### Train & Validation Set loader

In [25]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, input_shape, invert=False, aug='normal'):
        self.data_dir = data_dir
        # self.mode = mode
        self.input_shape = input_shape
        self.invert = invert
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        # if self.mode == 'train':
        #     self.db = self.db[:int(len(self.db) * 0.9)]
        # elif self.mode == 'val':
        #     self.db = self.db[int(len(self.db) * 0.9):]
        #     self.db.reset_index(inplace=True)
        # else:
        #     print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        if aug=='normal' :
            self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                                 transforms.Grayscale(),
                                                 transforms.ToTensor()]
                                                ) # [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
        elif aug=='horizon' :
            self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                                 transforms.RandomHorizontalFlip(p=1.0),
                                                 transforms.Grayscale(),
                                                 transforms.ToTensor()]
                                                )
    

    def data_loader(self):
        print('Loading dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))
        if self.invert :
            trans_image = 1 - trans_image

        return trans_image, torch.FloatTensor([1,0]) if data['COVID']==0 else torch.FloatTensor([0,1])


## Model

In [28]:
resnet = models.resnet50(pretrained=False)

In [36]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [29]:
resnet.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [30]:
resnet.fc = nn.Sequential(
            nn.Linear(2048,1024,bias=True),
            nn.ReLU(),
            nn.Dropout(0.7),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.7),
            nn.Linear(256,2),
            nn.Softmax(dim=1)
        )

## Utils
### EarlyStopper

In [6]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            self.save_model = False
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)

### Trainer

In [7]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            loss = self.loss_fn(pred, label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            target_lst.extend(label.argmax(dim=1).cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        wandb.log({'train_acc':self.train_score,'train_loss':self.train_mean_loss})
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred, label)
            val_total_loss += loss.item()
            target_lst.extend(label.argmax(dim=1).cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        wandb.log({'validation_acc':self.validation_score,'validation_loss':self.val_mean_loss})
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

### Metrics

In [8]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [26]:
augmentation = ['normal','horizon']
d_set = []
for c in augmentation:
    d = CustomDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE, aug=c)
    d_invert = CustomDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE, invert=True, aug=c)
    d_set.append(d)
    d_set.append(d_invert)

Loading dataset..
Loading dataset..
Loading dataset..
Loading dataset..


In [30]:
dataset = torch.utils.data.ConcatDataset(d_set)
len(dataset)

2584

In [53]:
N = len(dataset)
n = len(dataset)//25

In [56]:
train_dataset = torch.utils.data.Subset(df,range(n,N))
valid_dataset = torch.utils.data.Subset(df,range(n))

In [59]:
# Load dataset & dataloader
# train_dataset = torch.utils.data.ConcatDataset(d_set)
# validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(valid_dataset))

Train set samples: 2481 Val set samples: 103


In [71]:
from torchvision.utils import save_image
save_image(1-train_dataset[16][0],'bbb.png')

#### Load model and other utils

In [31]:
# Load Model
model = resnet.to(DEVICE)

# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn


# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

### epoch 단위 학습 진행

In [ ]:
wandb.init(entity='pnm-team',
           project='week1_Image',
           name='ResNet50',
           config={'learning_rate':5*1e-4,
                   'batch_size':32,
                   'epochs':40,
                   'input_size':255,
                   'Augmentation': ['Grayscale','Invert','RandomHorizontalFlip']},
           group='resnet50'
           )
for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)

    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, 'ResNet50.pt')
wandb.finish()

  0% 0/40 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.7160826606882943, Acc: 0.5073149741824441, F1-Macro: 0.4968604118439307


  2% 1/40 [04:29<2:55:28, 269.96s/it]

Epoch 0, Val loss: 1.0415975153446198, Acc: 0.5692307692307692, F1-Macro: 0.5190274841437632
Epoch 1, Train loss: 0.712924198144012, Acc: 0.5374354561101549, F1-Macro: 0.5252047427452756


  5% 2/40 [08:56<2:50:14, 268.80s/it]

Epoch 1, Val loss: 1.126614898443222, Acc: 0.5538461538461539, F1-Macro: 0.45219412961348443
Early stopping counter 1/15
Epoch 2, Train loss: 0.7126555318633715, Acc: 0.5370051635111877, F1-Macro: 0.5150953860234946
Epoch 2, Val loss: 0.90703184902668, Acc: 0.5384615384615384, F1-Macro: 0.5357142857142857
Validation loss decreased 1.0415975153446198 -> 0.90703184902668


  8% 3/40 [13:25<2:45:49, 268.89s/it]

## Inference
### 모델 로드

In [21]:
TRAINED_MODEL_PATH = 'ResNet50.pt'

### Load dataset

In [22]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.Grayscale(),
                                             transforms.ToTensor()]
                                            )

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [23]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

Loading test dataset..


### 추론 진행

In [24]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())

1it [00:08,  8.62s/it]

tensor([[9.1428e-01, 8.5719e-02],
        [9.8925e-01, 1.0752e-02],
        [8.2241e-01, 1.7759e-01],
        [3.0558e-01, 6.9442e-01],
        [9.8530e-01, 1.4699e-02],
        [3.5177e-02, 9.6482e-01],
        [2.2764e-01, 7.7236e-01],
        [4.0000e-01, 6.0000e-01],
        [6.8716e-01, 3.1284e-01],
        [6.8491e-01, 3.1509e-01],
        [4.4902e-01, 5.5098e-01],
        [9.5279e-01, 4.7207e-02],
        [9.1654e-01, 8.3459e-02],
        [4.7627e-02, 9.5237e-01],
        [2.2788e-01, 7.7212e-01],
        [6.7430e-01, 3.2570e-01],
        [8.1938e-01, 1.8062e-01],
        [9.9225e-01, 7.7486e-03],
        [3.2221e-01, 6.7779e-01],
        [8.7928e-01, 1.2072e-01],
        [9.8891e-01, 1.1088e-02],
        [3.1663e-01, 6.8337e-01],
        [9.1482e-01, 8.5180e-02],
        [9.1075e-01, 8.9245e-02],
        [9.7833e-03, 9.9022e-01],
        [9.8041e-01, 1.9589e-02],
        [9.9016e-01, 9.8436e-03],
        [9.5994e-01, 4.0065e-02],
        [1.6106e-01, 8.3894e-01],
        [4.138

### 결과 저장

In [25]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('submission_v009.csv', index=False)

In [26]:
df

,file_name,COVID
0,0.png,0
1,1.png,0
2,2.png,0
3,3.png,1
4,4.png,0
...,...,...
95,95.png,0
96,96.png,1
97,97.png,1
98,98.png,1
